In [1]:
import numpy as np
import pandas as pd

from Utils.data_proccessing import get_historical_ohlc
from pybit.unified_trading import HTTP
from constants import API_KEY, API_SECRET
from Utils.plotting import plotting_chart_technicals

In [2]:
session = HTTP(demo=True, api_key=API_KEY, api_secret=API_SECRET)
ohlc_m15 = get_historical_ohlc(session=session, symbol='BTCUSDT', interval='15')
ohlc_m5 = get_historical_ohlc(session=session, symbol='BTCUSDT', interval='5')

In [3]:
def detect_fvg(last_3_candels):

    fvg = 0
    fvg_top = 0
    fvg_bottom = 0

    high = last_3_candels['high'].values
    low = last_3_candels['low'].values

    if low[-1] > high[0]:

        fvg = 1
        fvg_top = low[-1]
        fvg_bottom = high[0]

    elif high[-1] < low[0]:
        
        fvg = -1
        fvg_top = low[0]
        fvg_bottom = high[-1]

    return fvg, fvg_top, fvg_bottom

In [4]:
def detect_fractal(last_3_candels):
    
    fractal = 0
    fractal_level = 0

    high = last_3_candels['high'].values
    low = last_3_candels['low'].values

    if high[-2] > high[-1] and high[-2] > high[-3]:

        fractal += 1
        fractal_level = high[-2]

    if low[-2] < low[-1] and low[-2] < low[-3]:

        fractal -= 1
        fractal_level = low[-2]

    return fractal, fractal_level

In [5]:
all_fractal = [0,0]
all_fractal_levels = [0,0]

for i in range(2, len(ohlc_m5)):
    
    last_3_candels = ohlc_m5.iloc[i-2:i+1]
    fractal, fractal_level = detect_fractal(last_3_candels)
    all_fractal.append(fractal)
    all_fractal_levels.append(fractal_level)

all_fractal = pd.Series(all_fractal, name='fractal')
all_fractal_levels = pd.Series(all_fractal_levels, name='fractal_level')

ohlc_m5 = pd.concat([ohlc_m5, all_fractal, all_fractal_levels], axis=1)

In [6]:
all_fvg = [0,0]
all_fvg_top = [0,0]
all_fvg_bottom = [0,0]

for i in range(2, len(ohlc_m15)):
    
    last_3_candels = ohlc_m15.iloc[i-2:i+1]
    fvg, fvg_top, fvg_bottom = detect_fvg(last_3_candels)
    all_fvg.append(fvg)
    all_fvg_top.append(fvg_top)
    all_fvg_bottom.append(fvg_bottom)

all_fvg = pd.Series(all_fvg, name='fvg')
all_fvg_top = pd.Series(all_fvg_top, name='fvg_top')
all_fvg_bottom = pd.Series(all_fvg_bottom, name='fvg_bottom')


ohlc_m15 = pd.concat([ohlc_m15, all_fvg, all_fvg_top, all_fvg_bottom], axis=1)

In [7]:
ohlc_m5.tail(10)

,time,open,high,low,close,volume,fractal,fractal_level
2490,2025-10-07 07:35:00,123720.8,123720.8,123580.0,123621.8,232.849,0,0.0
2491,2025-10-07 07:40:00,123621.8,123669.9,123500.0,123633.6,555.804,0,0.0
2492,2025-10-07 07:45:00,123633.6,123646.3,123560.4,123617.3,121.546,-1,123500.0
2493,2025-10-07 07:50:00,123617.3,123649.7,123570.5,123642.3,97.865,0,0.0
2494,2025-10-07 07:55:00,123642.3,123642.4,123522.8,123614.0,238.917,1,123649.7
2495,2025-10-07 08:00:00,123614.0,123623.7,123500.0,123535.9,130.179,0,0.0
2496,2025-10-07 08:05:00,123535.9,123627.8,123450.0,123624.1,377.812,0,0.0
2497,2025-10-07 08:10:00,123624.1,123650.0,123454.6,123494.6,357.865,-1,123450.0
2498,2025-10-07 08:15:00,123494.6,123572.9,123380.0,123392.3,281.845,1,123650.0
2499,2025-10-07 08:20:00,123392.3,123398.5,123200.1,123385.4,1214.063,0,0.0


In [8]:
ohlc_m15.tail(10)

,time,open,high,low,close,volume,fvg,fvg_top,fvg_bottom
2490,2025-10-07 06:00:00,124333.0,124398.0,124219.6,124225.8,206.466,0,0.0,0.0
2491,2025-10-07 06:15:00,124225.8,124252.6,123737.8,123820.8,1943.430,0,0.0,0.0
2492,2025-10-07 06:30:00,123820.8,123949.5,123716.9,123793.6,1533.660,-1,124219.6,123949.5
2493,2025-10-07 06:45:00,123793.6,123934.7,123740.0,123840.7,550.978,0,0.0,0.0
2494,2025-10-07 07:00:00,123840.7,124000.8,123745.9,123753.4,566.986,0,0.0,0.0
2495,2025-10-07 07:15:00,123753.4,123808.0,123520.0,123808.0,861.402,0,0.0,0.0
2496,2025-10-07 07:30:00,123808.0,123808.0,123500.0,123633.6,968.675,0,0.0,0.0
2497,2025-10-07 07:45:00,123633.6,123649.7,123522.8,123614.0,458.328,0,0.0,0.0
2498,2025-10-07 08:00:00,123614.0,123650.0,123450.0,123494.6,865.856,0,0.0,0.0
2499,2025-10-07 08:15:00,123494.6,123572.9,123200.1,123385.4,1495.297,0,0.0,0.0


In [9]:
def find_bos_from_fractals(df: pd.DataFrame,
                           mode: str = 'close',
                           max_lookahead: int | None = len(ohlc_m5)) -> pd.DataFrame:
    """
    Повертає копію df з додатковими колонками для рядків-фракталів:
      - fractal_price    : значення fractal_level у рядку фракталу
      - bos_close_price  : close свічки, що зробила BOS (NaN якщо не знайдено)
      - bos_index        : індекс (label) цієї свічки (NaN якщо не знайдено)
      - bos_time         : time цієї свічки (NaN якщо не знайдено)

    mode:
      - 'close' : вважаємо BOS коли close > fractal_level (для fractal==1)
                  або close < fractal_level (для fractal==-1)
      - 'body'  : вважаємо BOS коли мін(open,close) > fractal_level (для fractal==1)
                  або макс(open,close) < fractal_level (для fractal==-1)
    max_lookahead: максимальна кількість свічок у майбутнє (None -> до кінця)
    """
    df = df.copy()

    df['fractal_price'] = np.nan
    df['bos_close_price'] = np.nan
    df['bos_index'] = np.nan
    df['bos_time'] = pd.NaT

    # Індекси де є фрактали (і fractal_level не NaN)
    fractal_mask = df['fractal'].notna() & df['fractal_level'].notna()
    fractal_idxs = list(df.index[fractal_mask])

    pos_of_index = {idx: pos for pos, idx in enumerate(df.index)}
    n = len(df)

    for idx in fractal_idxs:
        f = df.at[idx, 'fractal']
        lvl = df.at[idx, 'fractal_level']
        df.at[idx, 'fractal_price'] = lvl

        start_pos = pos_of_index[idx] + 1
        if max_lookahead is None:
            end_pos = n
        else:
            end_pos = min(n, start_pos + int(max_lookahead))

        # Проходимо по майбутніх свічках
        found = False
        for pos in range(start_pos, end_pos):
            j = df.index[pos]
            o = float(df.iat[pos, df.columns.get_loc('open')])
            c = float(df.iat[pos, df.columns.get_loc('close')])

            if f == 1:
                if mode == 'close':
                    cond = c > lvl
                elif mode == 'body':
                    cond = min(o, c) > lvl
                else:
                    raise ValueError("mode must be 'close' or 'body'")
            elif f == -1:
                if mode == 'close':
                    cond = c < lvl
                elif mode == 'body':
                    cond = max(o, c) < lvl
                else:
                    raise ValueError("mode must be 'close' or 'body'")
            else:
                cond = False

            if cond:
                df.at[idx, 'bos_close_price'] = c
                df.at[idx, 'bos_index'] = j
                df.at[idx, 'bos_time'] = df.at[j, 'time'] if 'time' in df.columns else pd.NaT
                found = True
                break
    return df

ohlc_m5 = find_bos_from_fractals(ohlc_m5, mode='close', max_lookahead=len(ohlc_m5))
ohlc_m5.tail(15)

,time,open,high,low,close,volume,fractal,fractal_level,fractal_price,bos_close_price,bos_index,bos_time
2485,2025-10-07 07:10:00,123842.4,123881.8,123745.9,123753.4,187.847,1,124000.8,124000.8,NaN,NaN,NaT
2486,2025-10-07 07:15:00,123753.4,123767.6,123520.0,123724.3,619.469,0,0.0,0.0,NaN,NaN,NaT
2487,2025-10-07 07:20:00,123724.3,123794.9,123662.3,123736.0,133.498,-1,123520.0,123520.0,123494.6,2497.0,2025-10-07 08:10:00
2488,2025-10-07 07:25:00,123736.0,123808.0,123686.5,123808.0,108.435,0,0.0,0.0,NaN,NaN,NaT
2489,2025-10-07 07:30:00,123808.0,123808.0,123700.0,123720.8,180.022,0,0.0,0.0,NaN,NaN,NaT
2490,2025-10-07 07:35:00,123720.8,123720.8,123580.0,123621.8,232.849,0,0.0,0.0,NaN,NaN,NaT
2491,2025-10-07 07:40:00,123621.8,123669.9,123500.0,123633.6,555.804,0,0.0,0.0,NaN,NaN,NaT
2492,2025-10-07 07:45:00,123633.6,123646.3,123560.4,123617.3,121.546,-1,123500.0,123500.0,123494.6,2497.0,2025-10-07 08:10:00
2493,2025-10-07 07:50:00,123617.3,123649.7,123570.5,123642.3,97.865,0,0.0,0.0,NaN,NaN,NaT
2494,2025-10-07 07:55:00,123642.3,123642.4,123522.8,123614.0,238.917,1,123649.7,123649.7,NaN,NaN,NaT


In [ ]:
def find_fvg_fractal_bos_by_close(ohlc_m15, ohlc_m5, max_lookahead, debug: bool = False):
    """
    Шукає пари фракталів після M15 FVG і першу M5-свічку ПО ЧАСУ після f2,
    у якої close > f1_price (long) або close < f1_price (short).
    Повертає (signals_df, m5_out) — signals і m5 з мітками (в оригінальному індексі).
    """

    m15 = ohlc_m15.copy()
    m5_orig = ohlc_m5.copy()
    m15['time'] = pd.to_datetime(m15['time'])
    m5_orig['time'] = pd.to_datetime(m5_orig['time'])

    m5_orig = m5_orig.reset_index().rename(columns={'index': '_orig_index'})

    # Робоча версія з позиційним індексом 0..N-1
    m5 = m5_orig.sort_values('time').reset_index(drop=True)

    m5['is_f1'] = 0
    m5['is_f2'] = 0
    m5['is_bos'] = 0
    m5['source_fvg_time'] = pd.NaT

    signals = []
    n_m5 = len(m5)

    for _, fvg in m15[m15['fvg'] != 0].iterrows():
        fvg_dir = int(fvg['fvg'])
        fvg_time = fvg['time']
        fvg_top = float(fvg['fvg_top'])
        fvg_bottom = float(fvg['fvg_bottom'])

        m5_after = m5[m5['time'] > fvg_time]
        if m5_after.empty:
            continue

        # список фракталів (тільки реальні: !=0 і не NaN)
        fracs = m5_after[(m5_after['fractal'] != 0) & (~m5_after['fractal'].isna())]
        if len(fracs) < 2:
            continue

        # позиційні індекси
        frac_positions = list(fracs.index)

        # перебираємо сусідні пари (p1, p2)
        for i in range(len(frac_positions) - 1):
            p1 = frac_positions[i]
            p2 = frac_positions[i + 1]
            r1 = m5.loc[p1]
            r2 = m5.loc[p2]

            if fvg_dir == 1:
                # long: f1 == 1 та f1.price > fvg_top
                #       f2 == -1 та f2.price < fvg_top і > fvg_bottom
                cond_pair = (r1['fractal'] == 1) and (float(r1['fractal_level']) > fvg_top) \
                            and (r2['fractal'] == -1) and (float(r2['fractal_level']) < fvg_top) and (float(r2['fractal_level']) > fvg_bottom)
            else:
                # short: f1 == -1 та f1.price < fvg_top
                #        f2 == 1 та f2.price > fvg_top і < fvg_bottom
                cond_pair = (r1['fractal'] == -1) and (float(r1['fractal_level']) < fvg_bottom) \
                            and (r2['fractal'] == 1) and (float(r2['fractal_level']) > fvg_bottom) and (float(r2['fractal_level']) < fvg_top)

            if not cond_pair:
                continue

            # починаємо шукати BOS з наступної свічки після f2
            start_pos = p2
            if start_pos >= n_m5:
                break

            f1_price = float(r1['fractal_level'])
            bos_pos = None

            end_pos = n_m5 if max_lookahead is None else min(n_m5, start_pos + int(max_lookahead))

            for pos in range(start_pos, end_pos):
                close = float(m5.at[pos, 'close'])
                if fvg_dir == 1:
                    cond = close > f1_price   # long: close вище f1
                elif fvg_dir == -1:
                    cond = close < f1_price   # short: close нижче f1
                else:
                    return None

                if cond:
                    bos_pos = pos
                    break

            if bos_pos is not None:
                sig = {
                    'fvg_dir': fvg_dir,
                    'fvg_time': fvg_time,
                    'fvg_top': fvg_top,
                    'fvg_bottom': fvg_bottom,
                    'f1_pos': int(p1),
                    'f1_time': r1['time'],
                    'f1_price': float(r1['fractal_level']),
                    'f2_pos': int(p2),
                    'f2_time': r2['time'],
                    'f2_price': float(r2['fractal_level']),
                    'bos_pos': int(bos_pos),
                    'bos_time': m5.at[bos_pos, 'time'],
                    'bos_close': float(m5.at[bos_pos, 'close'])
                }
                signals.append(sig)

                m5.at[p1, 'is_f1'] = 1
                m5.at[p2, 'is_f2'] = 1
                m5.at[bos_pos, 'is_bos'] = 1
                m5.at[p1, 'source_fvg_time'] = fvg_time
                m5.at[p2, 'source_fvg_time'] = fvg_time
                m5.at[bos_pos, 'source_fvg_time'] = fvg_time

                # if debug:
                #     print("FVG", fvg_time, "paired f1@pos", p1, "f2@pos", p2, "-> bos@pos", bos_pos)
                # break   

    signals_df = pd.DataFrame(signals)
    if '_orig_index' in m5.columns:
        m5_out = m5.set_index('_orig_index').sort_index()
    else:
        if debug:
            print("Warning: '_orig_index' not in m5.columns; returning positional-index m5")
        m5_out = m5

    return signals_df, m5_out

signals, m5_annot = find_fvg_fractal_bos_by_close(ohlc_m15, ohlc_m5, max_lookahead=len(ohlc_m5))

signals.tail()

,fvg_dir,fvg_time,fvg_top,fvg_bottom,f1_pos,f1_time,f1_price,f2_pos,f2_time,f2_price,bos_pos,bos_time,bos_close
163,-1,2025-10-06 21:30:00,124982.8,124948.3,2371,2025-10-06 21:40:00,124566.0,2373,2025-10-06 21:50:00,124948.4,2414,2025-10-07 01:15:00,124545.6
164,-1,2025-10-06 23:15:00,124829.5,124811.8,2392,2025-10-06 23:25:00,124604.9,2394,2025-10-06 23:35:00,124813.8,2403,2025-10-07 00:20:00,124593.4
165,1,2025-10-07 04:30:00,124324.0,124247.0,2462,2025-10-07 05:15:00,124469.7,2463,2025-10-07 05:20:00,124290.6,2466,2025-10-07 05:35:00,124509.8
166,1,2025-10-07 04:45:00,124417.0,124352.0,2465,2025-10-07 05:30:00,124508.3,2466,2025-10-07 05:35:00,124357.2,2466,2025-10-07 05:35:00,124509.8
167,-1,2025-10-07 06:30:00,124219.6,123949.5,2484,2025-10-07 07:05:00,123817.9,2485,2025-10-07 07:10:00,124000.8,2485,2025-10-07 07:10:00,123753.4


In [11]:
def get_signal(current_candle):

    signal = 0
    stop_loss = 0
    take_profit = 0

    if current_candle['fvg'] == 1:
        signal = 1
        stop_loss = current_candle['fvg_bottom']
        take_profit = current_candle['close'] + [current_candle['close'] - stop_loss] * 2
    elif current_candle['fvg'] == -1:
        signal = -1
        stop_loss = current_candle['fvg_top']
        take_profit = current_candle['close'] - (abs(current_candle['close'] - stop_loss)) * 2
    return signal, stop_loss, take_profit

In [12]:
all_signals = []
all_sls = []
all_tps = []

for i in range(len(ohlc)):

    current_candle = ohlc.iloc[i]
    signal, stop_loss, take_profit = get_signal(current_candle=current_candle)
    all_signals.append(signal)
    all_sls.append(stop_loss)
    all_tps.append(take_profit)

all_signals = pd.Series(all_signals, name='signal')
all_sls = pd.Series(all_sls, name='stop_loss')
all_tps = pd.Series(all_tps, name='take_profit')

df = pd.concat([ohlc, all_signals, all_sls, all_tps], axis=1)
df['take_profit'] = pd.to_numeric(df['take_profit'], errors='coerce')
df['stop_loss'] = pd.to_numeric(df['stop_loss'], errors='coerce')
df

NameError: name 'ohlc' is not defined

In [ ]:
print(df.dtypes)
print(type(df.iloc[0]['high']), df.iloc[0]['high'])

In [ ]:
def run_backtest(df):

    LONG = 1
    SHORT = -1
    
    current_trades = {}
    trades_data = {}

    trade_open = False

    if len(df) > 0 and len(df[df['signal'] != 0]) > 0:

        for i in range(len(df) - 1):

            cc = df.iloc[i]
            nc = df.iloc[i+1]
            

            if cc['signal'] and not trade_open:

                direction = LONG if cc['signal'] == LONG else SHORT

                current_trades[len(current_trades)] = {
                    'direction': direction,
                    'entry_level': nc['open'],
                    'stop_loss': cc['stop_loss'],
                    'take_profit': cc['take_profit'],
                    'entry_index': i
                    }
                
                trade_open = True

            if trade_open:

                trades_to_remove = []

                for n_of_trade, current_trade in current_trades.items():

                    direction = current_trade['direction']
                    entry =  current_trade['entry_level']
                    sl = current_trade['stop_loss']
                    tp = current_trade['take_profit']
                    profit = None

                    if direction == LONG:

                        if (nc['low'] <= sl and nc['high'] >= tp or 
                            nc['low'] <= sl):

                            profit = sl - entry

                        elif nc['high'] > tp:

                            profit = tp - entry
                    
                    elif direction == SHORT:

                        if (nc['high'] >= sl and nc['low'] <= tp or 
                            nc['high'] >= sl):

                            profit = entry - sl

                        elif nc['low'] < tp:
                            profit = entry - tp

                    if profit is not None:
                        trades_data[len(trades_data)] = {
                            'direction': direction,
                            'entry_level': entry,
                            'stop_loss': sl,
                            'take_profit': tp,
                            'profit': profit,
                            'entry_index': current_trade['entry_index'],
                            'exit_index': i + 1
                            }
                        
                        trade_open = False
                        trades_to_remove.append(n_of_trade)
                
                if len(trades_to_remove) > 0:
                    for trade in trades_to_remove:
                        current_trades.pop(trade, None)

    return pd.DataFrame(trades_data).transpose()

In [ ]:
trades = run_backtest(df)

In [ ]:
trades

In [ ]:
MAX_LOOKAHEAD = len(ohlc)

# Ініціалізація (скидаємо старі значення)
ohlc['bos_flag'] = 0
ohlc['bos_index'] = -1   # зберігатимемо глобальну позицію свічки, де відбувся BOS
ohlc['bos_time'] = pd.NaT
ohlc['bos_type'] = 0
ohlc['bars_to_bos'] = np.nan
ohlc['bos_price'] = np.nan

# кеш масивів для швидкості
n = len(ohlc)
closes = ohlc['close'].values
fractals = ohlc['fractal'].fillna(0).values
fr_levels = ohlc['fractal_level'].fillna(0).values

last_high = None   # позиція останнього хай-фракталу (індекс у діапазоні 0..n-1)
last_low = None    # позиція останнього лоу-фракталу

for pos in range(n):
    # якщо ця свічка є фракталом — оновлюємо останній відповідного типу
    fr = int(fractals[pos])
    if fr == 1:
        last_high = pos
    elif fr == -1:
        last_low = pos

    # перевіряємо, чи стався пробій по high-фракталу (тобто close > level)
    if last_high is not None:
        lvl = fr_levels[last_high]
        if lvl != 0 and closes[pos] > lvl:
            # проставляємо BOS тільки в рядку самого фракталу last_high
            idx = ohlc.index[last_high]
            ohlc.at[idx, 'bos_flag'] = 1
            ohlc.at[idx, 'bos_index'] = pos            # глобальна позиція свічки пробою
            ohlc.at[idx, 'bos_time'] = ohlc.index[pos]
            ohlc.at[idx, 'bos_type'] = 1
            ohlc.at[idx, 'bars_to_bos'] = pos - last_high
            ohlc.at[idx, 'bos_price'] = closes[pos]
            # затираємо last_high — щоб старіші high-фрактали не отримали BOS
            last_high = None

    # аналогічно для low-фракталу (пробій вниз)
    if last_low is not None:
        lvl = fr_levels[last_low]
        if lvl != 0 and closes[pos] < lvl:
            idx = ohlc.index[last_low]
            ohlc.at[idx, 'bos_flag'] = 1
            ohlc.at[idx, 'bos_index'] = pos
            ohlc.at[idx, 'bos_time'] = ohlc.index[pos]
            ohlc.at[idx, 'bos_type'] = -1
            ohlc.at[idx, 'bars_to_bos'] = pos - last_low
            ohlc.at[idx, 'bos_price'] = closes[pos]
            last_low = None

ohlc.tail(15)



In [ ]:
plotting_chart_technicals(ohlc, 100, 150)

In [ ]:
df = pd.read_csv('/Users/synyshyn_04/BTC_backtest_new/all_results.csv')
df